In [1]:
import requests
import pprint
import json
import pandas as pd
import math

In [2]:
perpage=200

In [3]:
#To do: 
#  Trier par type de publication, 
#  Ajouter critère auteur/journal/institution
#  Gestion des OU

my_search = input("What would you like to search for? (Expected format: Title(Keyword1' AND/OR 'Keyword2' ..) AND/OR Abstract(Keyword1' AND/OR 'Keyword2' ..) AND SORTBY(Date/NbCitation))")
customName = "MySearch"
keywords_title = []
keywords_abstract =  ["presence","EEG","Virtual","Reality"]

In [34]:
def prodUrl(keywords_title,keywords_abstract,page=0,perpage=perpage):
    request = "https://api.openalex.org/works?"
    req=request
    search = "per-page="+str(perpage)+"&"
    req = req+search
    if(page!=0):
        search = "page="+str(page)+"&"
    req = req+search
    req = req+"filter="
    if(keywords_title!=[]):
        search = "title.search:"
        for i in range(0,len(keywords_title)):
            if(i != len(keywords_title)-1):
                search+=keywords_title[i]+"%26"
            else:
                search+=keywords_title[i]
        req = req+search+','
    search = "abstract.search:"
    for i in range(0,len(keywords_abstract)):
        if(i != len(keywords_abstract)-1):
            search+=keywords_abstract[i]+"%26"
        else:
            search+=keywords_abstract[i]
    req = req+search+','
    
    if(req[-1]==','):
        req=req[:-1]
    return req
def getResults(req):
    response = requests.get(url=req)
    response.raise_for_status()
    random_work = response.json()
    return random_work

def saveResults(random_work,name):
    with open("sample.json", "w") as outfile:
        json.dump(random_work, outfile)
    
    df = pd.read_json ('sample.json')
    df.to_csv (name+".csv", index = None, sep=';',header=True)
    
def correctionCSVfile(name):
    with open(name+".csv", "r+") as f:
        first_line = f.readline()
        if first_line != "sep=,":
            lines = f.readlines()
            f.seek(0)
            f.write("sep=,")
            f.write(first_line)
            f.writelines(lines)       
        
def gatherFiles(customName,keywords_title,keywords_abstract):
    url = prodUrl(keywords_title,keywords_abstract)
    res = getResults(url)
    nb_results = res['meta']['count']
    nb_page = math.ceil(nb_results/perpage)
    results = []
    for i in range(0,nb_page):
        url = prodUrl(keywords_title,keywords_abstract,page=i)
        res = getResults(url)
        results+=res['results']
    saveResults(results,customName)

In [35]:
gatherFiles(customName,keywords_title,keywords_abstract)